# Titanic 

Classify whether a passenger survived or perished in the Titanic sinking. Using a least-squares classifier to analyze model predicitons.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

The `load_clean` function performs feature engineering. Some interesting findings are:
- `Alone`: much more likely to perish if alone
- `Child`: those under age 6 had a higher survival chance.

In [ ]:
def load_clean(input_dir):
    data = pd.read_csv(input_dir)
    data = data.set_index("PassengerId")
    data["Embarked"] = data["Embarked"].replace({
        "C": "Cherbourg", 
        "Q": "Queenstown", 
        "S": "Southampton"
    })
    try:
        data["target"] = data["Survived"].replace({0:-1})
        data["Survived"] = data["Survived"].replace({0:"No", 1:"Yes"})
    except:
        # test data
        pass
    data["Pclass"] = data["Pclass"].replace({
        1: "1st_Class", 2: "2nd_Class", 3: "3rd_Class"
    })
    data["Alone"] = data[["SibSp", "Parch"]].sum(axis=1) == 0
    data["LastName"] = data["Name"].str.split(",", 1).str[0]
    last_names = data["LastName"].sort_values().value_counts()
    last_names.name = "LastNameCount"
    last_names = pd.DataFrame([last_names]).T
    last_names["FamilyGroup"] = np.arange(1, len(last_names)+1)
    last_names.loc[last_names["LastNameCount"] == 1, "FamilyGroup"] = 0
    data = data.merge(last_names, left_on="LastName", right_index=True).sort_index()
    cabins = data["Cabin"].str.split("\d").str[0]
    data["Deck"] = cabins[cabins.notna()]
    return data

input_dir = "../input/titanic/train.csv"
data = load_clean(input_dir)
N = data.shape[0]
print("Number of observations:", N)
survived = (data["target"] == 1).sum()
survive_pct = survived / N
print(f"Survival percentage {survived/N:.1%} = ({survived} / {N}).\nDead {1-survived/N:.1%} = ({N - survived} / {N})")

In [ ]:
test_dir = "../input/titanic/test.csv"
test_data = load_clean(test_dir)

In [ ]:
sns.histplot(data, x="Age", hue="Survived", bins=50)

In [ ]:
sns.countplot(data=data, x="Pclass", hue="Survived");

# Modeling

Trying out a least-squares classifier. Predict survived $\tilde y > 0$ where $\tilde y = X \beta + v$. The prediction takes the sign $\hat y = \textbf{sign}(\tilde y) \in \{ -1, 1\}$ for `No` or `Yes` Survived.

A model just including the `Pclass` just uses the proportion of each class as a predictor. On the histogram, the 3 bars represent each class. The far-right is 1st class, which has the highest proportion of survival. The middle class is just below 0, meaning 2nd class is more likely to die. Far-left near -0.5 is 3rd class, which has a large proportion of dead people. Adjusting the classification decision $\alpha$ changes the true and false positive rate, and traces out the ROC curve.

In [ ]:
def roc(ytilde, y):
    alphas = np.linspace(-1, 1)
    cnf_list = []
    for alpha in alphas:
        yhat = np.sign(ytilde - alpha)
        cnf = confusion_matrix(y, yhat, labels=[1, -1]).ravel()
        cnf_list.append(cnf)
    # (tp, fn, fp, tn)
    cnf = np.append(alphas[:, np.newaxis], np.array(cnf_list), axis=1)

    fprate = cnf[:,3] / cnf[:, [3,4]].sum(axis=1)
    tprate = cnf[:,1] / cnf[:, [1,2]].sum(axis=1)
    error = 1 - cnf[:, [1,4]].sum(axis=1) / cnf[:,1:].sum(axis=1)
    a_opt = alphas[error.argmin()]
    print(f"Accuracy: {1 - error.min():.1%}")
    print(f"Alpha: {a_opt: .3f}")

    fig, axs = plt.subplots(1, 3, tight_layout=True, figsize=(12, 4))
    sns.histplot(x=ytilde, hue=data["Survived"], ax=axs[0], bins=30)
    axs[0].set_xlabel(r"$\tilde f(x^{(i)})$")
    axs[0].axvline(a_opt, ls="--", color="k")

    axs[1].plot(cnf[:,0], tprate,   label="True Positive Rate")
    axs[1].plot(cnf[:,0], fprate,   label="False Positive Rate")
    axs[1].plot(cnf[:,0], error,    label="Error Rate")
    axs[1].axvline(a_opt, ls="--", color="k")
    axs[1].legend(loc="upper right")
    axs[1].set_xlabel(r"$\alpha$")
    axs[1].set_ylabel("Rate")
   
    axs[2].plot(fprate, tprate)
    axs[2].set_xlabel("False Positive Rate")
    axs[2].set_ylabel("True Positive Rate")

    return cnf

In [ ]:
X = pd.get_dummies(data["Pclass"], drop_first=True)
# X["Fare"] = preprocessing.scale(data["Fare"])
y = data["target"]
lm = linear_model.LinearRegression(fit_intercept=True)
lm.fit(X, y)
ytilde = lm.predict(X)
cnf = roc(ytilde, y)

A more expressive model includes `Alone`, `Child`, and `Sex` which increases prediction accuracy to over 80%. Adding more features does not improve accuracy very much.

In [ ]:
rng = np.random.default_rng(0)

X1 = X.copy()
X1["Alone"] = data["Alone"].astype(int)
X1["Child"] = (data["Age"] < 6).astype(int)             # look at histogram
X1["Sex"] = data["Sex"].replace({"female":0, "male":1}) # improves accuracy by 9%

lm.fit(X1, y)
ytilde = lm.predict(X1)
cnf = roc(ytilde, y)

In [ ]:
X2 = X1.copy()

X2["Fare"] = preprocessing.scale(data["Fare"])
X2["Teen"] = (data["Age"].between(6, 18)).astype(int)
X2 = X2.merge(pd.get_dummies(data["Deck"])[["A", "B", "C", "D", "E", "F"]], left_index=True, right_index=True)
X2["SibSp"] = data["SibSp"]
X2["Parch"] = data["Parch"]

lm.fit(X2, y)
ytilde = lm.predict(X2)
cnf = roc(ytilde, y)

Cross-validation with different models improves accuracy by a couple percentage points depending on classifier type (RandomForest, SVC).

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=6)
# classifier = svm.SVC(kernel="linear", probability=True, random_state=np.random.RandomState(0))
classifier = RandomForestClassifier()
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
for i, (train, test) in enumerate(cv.split(X2, y)):
    classifier.fit(X2.iloc[train], y.iloc[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X2.iloc[test], y.iloc[test],
        name="ROC fold {}".format(i),
        alpha=0.3,
        lw=1,
        ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0,1], [0,1], ls="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color="b",
        label=r"Mean ROS (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,
        color="grey", alpha=0.2, label=r"$\pm$ 1 std. dev.")

ax.set(
    xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
    title="ROC"
)
ax.legend(loc="lower right");

# Test

In [ ]:
lm.fit(X2, y)

Xt = pd.get_dummies(test_data["Pclass"], drop_first=True)
Xt["Fare"] = preprocessing.scale(test_data["Fare"])
Xt["Fare"] = Xt["Fare"].fillna(0)
Xt["Alone"] = test_data["Alone"].astype(int)
Xt["Teen"] = (test_data["Age"].between(6, 18)).astype(int)
Xt["Child"] = (test_data["Age"] < 6).astype(int)             # look at histogram
Xt["Sex"] = test_data["Sex"].replace({"female":0, "male":1}) # improves accuracy by 9%
Xt = Xt.merge(pd.get_dummies(test_data["Deck"])[["A", "B", "C", "D", "E", "F"]], left_index=True, right_index=True)
Xt["SibSp"] = test_data["SibSp"]
Xt["Parch"] = test_data["Parch"]

Xt = Xt.loc[:, X2.columns]

y_tilde = lm.predict(Xt)
y_pred = (y_tilde > 0).astype(int)
# y_pred = classifier.predict(Xt)
test_pred = pd.Series(y_pred, index=test_data.index)
test_pred.name="Survived"
test_pred.to_csv("submission.csv")